In [1]:
import requests
import json
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pandas as pd


In [2]:
# wait
def wait(driver, selector_name):
    while (True):
        try:
            myElem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, selector_name)))
            if myElem is not None:
                break
        except TimeoutException:
            break

In [3]:
# scroll
def scroll(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [4]:
main_url = "https://soundcloud.com/discover"
set_link = []

In [5]:
import sys

def printProgressBar(i,max,postText):
    n_bar =10 #size of progress bar
    j= i/max
    sys.stdout.write('\r')
    sys.stdout.write(f"[{'=' * int(n_bar * j):{n_bar}s}] {int(100 * j)}%  {postText}")
    sys.stdout.flush()

In [6]:
# Drive the chrome to url link and return soup with html parser
def drive_to_url_page(driver, url):
    driver.get(url)
    scroll(driver)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    return soup

In [7]:
#Driver to soundcloud website
driver = webdriver.Chrome()
soup = drive_to_url_page(driver, main_url)
links = soup.find_all("a", {"class": "sc-link-secondary sc-link-light playableTile__usernameHeading audibleTile__usernameHeading sc-truncate sc-text-h4"})
for i, item in enumerate(links):
    user_url = "https://soundcloud.com" + item.get("href")
    if user_url not in set_link:
        set_link.append(user_url)
    if len(set_link) >= 19:
            break
    
    printProgressBar(i, len(links), "'%'")
# #Get the all person in set_link with his/her following information
# for i in range(0, len(set_link) - 1):
#     following_url = set_link[i] + "/following"
#     soup = drive_to_url_page(driver, following_url)
#     link_following_user = soup.find_all("a", {"class":"userBadgeListItem__heading sc-type-small sc-text-h4 sc-link-dark sc-link-primary sc-truncate"})
#     for item in link_following_user:
#         user_url = "https://soundcloud.com" + item.get("href")
#         if user_url not in set_link:
#             set_link.append(user_url)
driver.close()

[========= ] 96%  '%'

In [8]:
len(set_link)

18

In [9]:
#Count all playlist in user_url/sets (count by <li> tag)
def count_playlist_in_one_person(driver, url_of_user):
    playlist_url = str(url_of_user) + '/sets'
    soup = drive_to_url_page(driver, playlist_url)
    playlist_count = soup.find_all("li", {"class":"soundList__item"})
    return len(playlist_count)

In [10]:
def get_all_user_data(driver, set_link):
    driver = webdriver.Chrome()    
    #Get user id in SoundCloud/Discover by get meta tag in HEAD html
    set_data_user = []
    for i, link_user in enumerate(set_link):
        one_data_user = []
        driver.get(link_user)
        wait(driver, 'infoStats__value sc-font-light')
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        
        #Get USER INFO
        id_user = soup.find("meta", property="twitter:app:url:googleplay")
        name_user = soup.find("meta", property="og:title")
        url_user = soup.find("meta", property="og:url")
        info_start = soup.find_all("div", {"class":"infoStats__value sc-font-light"})
        
        likeTrack_count = soup.find_all("span", {"class":"sidebarHeader__actualTitle sc-text-h3"})
        playlist_count = count_playlist_in_one_person(driver, link_user)
        # more_info = soup.find("h3", {"class": "profileHeaderInfo__additional g-type-shrinkwrap-block theme-dark g-type-shrinkwrap-large-secondary sc-mt-1x"}).getText().Strip()

        #Add to one_data_user set
        one_data_user.append(id_user["content"].strip("soundcloud://users:"))
        one_data_user.append(name_user["content"])
        one_data_user.append(url_user["content"])
        one_data_user.append(playlist_count)
        # one_data_user.append(more_info)
        one_data_user.append(info_start[0].getText())   #followers
        one_data_user.append(info_start[1].getText())   #following
        one_data_user.append(info_start[2].getText())   #Track count
        one_data_user.append(likeTrack_count[1].getText().strip(' likes'))
        #Add one data user to set data 
        set_data_user.append(one_data_user)
        
        printProgressBar(i, len(set_link), "'%'")
        
    driver.close()
    return set_data_user

In [11]:
set_data_user = get_all_user_data(driver, set_link)
len(set_data_user)

[========= ] 94%  '%'

18

In [12]:
#write set data user to file <'user.csv'>
df = pd.DataFrame(set_data_user,columns=['User ID', 
                                         'User Name', 
                                         'User Link', 
                                         'Playlist Count', 
                                         'Followers', 
                                         'Following', 
                                         'Track Count', 
                                         'Like Tracks'])
df.to_csv('userHTML.csv', index=False, sep='\t', encoding='utf-8')

In [13]:
set_tracks = []

In [14]:
def getTracks(set_user):
    driver = webdriver.Chrome()
    set_tracks = []
    for i, item in enumerate(set_user):
        temp = item + "/tracks"
        driver.get(temp)

        wait(driver, selector_name='infoStats__value sc-font-light')

        scroll(driver)

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        user_id = soup.find('meta', {'property': 'twitter:app:url:googleplay'})['content'].split(':')[-1]
        link_tracks = soup.find_all('a', {'class': 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'})
        link_tracks = link_tracks[:-3]
        info_start = soup.find_all("div", {"class": "infoStats__value sc-font-light"})
        track_count = info_start[2].getText()
        if track_count != '0':
            for link in link_tracks:
                string_temp = 'https://soundcloud.com'
                string_temp = string_temp + link['href']
                driver.get(string_temp)
                wait(driver, selector_name='sc-ministats-item')
                soup = BeautifulSoup(driver.page_source, "html.parser")
                id_track = soup.find('meta', {'property': 'twitter:app:url:googleplay'})['content'].split(':')[-1]
                name_track = soup.find('h1', {'class': 'soundTitle__title sc-font g-type-shrinkwrap-inline g-type-shrinkwrap-large-primary theme-dark'}).getText().strip()
                time_release_track = soup.find('time', {'class': 'relativeTime'})['title']
                info_track = soup.find_all('span', {'aria-hidden': 'true'})
                plays = info_track[1].getText()
                likes = info_track[2].getText()
                reposts = info_track[3].getText()

                tup = [user_id, id_track, name_track, time_release_track, plays, likes, reposts]
                set_tracks.append(tup)
        else:
            set_tracks.append([user_id, "None", "None", "None", "None", "None", "None"])
        
        printProgressBar(i, len(set_user), "'%'")
    driver.close()
    return set_tracks

In [15]:
set_tracks = getTracks(set_link)
len(set_tracks)

[========= ] 94%  '%'

76

In [16]:
df = pd.DataFrame(set_tracks, columns=['User ID',
                                      'Tracks ID',
                                      'Tracks Name',
                                      'Time_Release_Track',
                                      'Plays',
                                      'Likes',
                                      'Repost'])
df.to_csv('tracksHTML.csv', index=False, sep='\t', encoding='utf-8')

In [17]:
#Get all tracks in playlist 

In [18]:
set_playlists = []

In [ ]:
def getTracksFromPlaylists(playlist_url, optional):
    driver = webdriver.Chrome()
    driver.get(playlist_url)
    wait(driver, selector_name='trackItem__trackTitle')
    scroll(driver)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    user_name = soup.find("a", {"class": "userBadge__usernameLink"}).getText().strip()
    playlist_title = soup.find("h1", {"class": "soundTitle__title"}).getText().strip()
    num_of_track = soup.find("div", {"class": "genericTrackCount__title"}).getText()
    create_date = soup.find("time", {"class": "relativeTime"})["title"]
    datas = soup.find_all("span", {"aria-hidden": "true"})

    likes = datas[1].getText()
    reports = datas[2].getText()

    track_list = []
    track_str = []

    if optional == 0:
        track_items = soup.find_all("a", {"class": "trackItem__trackTitle sc-link-dark sc-link-primary sc-font-light"})
        for track_item in track_items:
            track_name = 'https://soundcloud.com' + track_item['href']
            track_list.append(track_name.strip().replace("\n", " "))
    elif optional == 1:
        track_items = soup.find_all("a", {"class": "trackItem__trackTitle"})
        track_urls = []

        for track_item in track_items:
            track_urls.append('https://soundcloud.com' + track_item['href'])

        for url in track_urls:
            driver.get(url)
            wait(driver, selector_name='soundTitle__usernameTitleContainer')
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            meta = soup.find("meta", property="twitter:app:url:iphone")
            track_list.append(meta["content"].split(':')[-1])

    track_str = ','.join(track_list)
    if track_str == '':
        track_str = 'None'
    
    driver.close()
    return [user_name, playlist_title, num_of_track, create_date, likes, reports, track_str]

In [35]:
def getPlaylistURLs(user_playlist_url):
    driver = webdriver.Chrome()
    driver.get(user_playlist_url)
    scroll(driver)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    contents = soup.find_all("a", {"class": "sound__coverArt"})
    urls = []

    for content in contents:
        urls.append('https://soundcloud.com' + content['href'])
    
    driver.close()
    return urls

In [36]:
def getPlaylists(user_playlist_urls, optional):
    playlist_lst = []
    for i, user_url in enumerate(user_playlist_urls):
        driver = webdriver.Chrome()
        driver.get(user_url)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        user_id = soup.find('meta', {'property': 'twitter:app:url:googleplay'})['content'].split(':')[-1]
        
        playlist_urls = getPlaylistURLs(user_url + '/sets')
        
        for j, playlist_url in enumerate(playlist_urls):
            playlist_lst.append(user_id + getTracksFromPlaylists(playlist_url, optional))
            printProgressBar(j, len(playlist_urls), '')
        
        printProgressBar(i, len(user_playlist_urls), "'%'")
    return playlist_lst

In [52]:
set_playlists = getPlaylists(set_link, optional=0)
len(set_playlists)

https://soundcloud.com/solotoko/sonny-fodera-dom-dolla-moving-blind-1?in=digitalstreams/sets/futurerave
https://soundcloud.com/camelphat/rabbit-hole?in=digitalstreams/sets/futurerave
https://soundcloud.com/bensonaus/benson-designer-girl?in=digitalstreams/sets/futurerave
https://soundcloud.com/davidguetta/kill-me-slow?in=digitalstreams/sets/futurerave
https://soundcloud.com/meduzamusic/piece-of-your-heart-feat?in=digitalstreams/sets/futurerave
https://soundcloud.com/halseymusic/graveyard-axwell-remix?in=digitalstreams/sets/futurerave
https://soundcloud.com/robin-schulz/robin-schulz-in-your-eyes-feat?in=digitalstreams/sets/futurerave
https://soundcloud.com/vintageculturemusic/maverick-sabre-slow-down-vintage-culture-slow-motion-remix?in=digitalstreams/sets/futurerave
https://soundcloud.com/valentinokhan/valentino-khan-blackmail?in=digitalstreams/sets/futurerave
https://soundcloud.com/kaskade/kaskade-cant-be-without?in=digitalstreams/sets/futurerave
https://soundcloud.com/chrislake/the-an

https://soundcloud.com/tiesto/in-my-mind-axwell-mix-ivan?in=digitalstreams/sets/top20
https://soundcloud.com/aviciiofficial/levels-original-version?in=digitalstreams/sets/top20
https://soundcloud.com/officialdigitalrecords/habibi?in=digitalstreams/sets/top20
https://soundcloud.com/steveaoki/kid-cudi-pursuit-of-happiness?in=digitalstreams/sets/top20
https://soundcloud.com/kaskade/lana-del-rey-young-and?in=digitalstreams/sets/top20
https://soundcloud.com/tommytrash/reload?in=digitalstreams/sets/top20
https://soundcloud.com/zedd/spectrum?in=digitalstreams/sets/top20
https://soundcloud.com/davidguetta/titanium-feat-sia?in=digitalstreams/sets/top20
https://soundcloud.com/steveangello/pendulum-the-island-steve-angello-an21-max-vangeli-remix?in=digitalstreams/sets/top20
https://soundcloud.com/stromae/alors-on-danse?in=digitalstreams/sets/top20
https://soundcloud.com/hardwell/rihanna-where-have-you-been?in=digitalstreams/sets/top20
https://soundcloud.com/tiesto/if-i-lose-myself-alesso-vs?in=di

https://soundcloud.com/spinninrecords/klosman-nico-de-andrea-porque-me-voy?in=digitalstreams/sets/best-spanish-house-vol-01
https://soundcloud.com/ninja-tune/peggy-gou-it-makes-you-forget-itgehane-2?in=digitalstreams/sets/best-spanish-house-vol-01
https://soundcloud.com/spinninrecords/gianluca-vacchi-viento?in=digitalstreams/sets/best-spanish-house-vol-01
https://soundcloud.com/viciousrecordings/avicii-malo-original-mix?in=digitalstreams/sets/best-spanish-house-vol-01
https://soundcloud.com/spinninrecords/alok-bhaskar-fuego?in=digitalstreams/sets/best-spanish-house-vol-01
https://soundcloud.com/deorro/bailar-feat-pitbull-elvis?in=digitalstreams/sets/best-spanish-house-vol-01
https://soundcloud.com/wsoundsfunk/mc-fioti-bum-bum-tam-tam?in=digitalstreams/sets/best-spanish-house-vol-01
https://soundcloud.com/cedricgervais/cedric-gervais-feat-juanes-este-amor?in=digitalstreams/sets/best-spanish-house-vol-01
https://soundcloud.com/dillonfrancis/dillon-francis-we-the-funk-feat-fuego-1?in=digi

https://soundcloud.com/penthouse-penthouse/let-me-see-that?in=digitalstreams/sets/sexydeephouse
https://soundcloud.com/crymeasong/chris-malinchak-call-my-name?in=digitalstreams/sets/sexydeephouse
https://soundcloud.com/thetransplant2edits/theres-something-about-us-under-the-sheets-ellie-goulding-x-daft-punk?in=digitalstreams/sets/sexydeephouse
https://soundcloud.com/stwosc/haunted-ft-sevdaliza?in=digitalstreams/sets/sexydeephouse
https://soundcloud.com/edmeden/mark-knight-your-love-original?in=digitalstreams/sets/sexydeephouse
https://soundcloud.com/jessieslife/draketoomuchremix?in=digitalstreams/sets/sexydeephouse
https://soundcloud.com/dariusofficial/hot-hands?in=digitalstreams/sets/sexydeephouse
https://soundcloud.com/futureclassic/touch-sensitive-slowments-3?in=digitalstreams/sets/sexydeephouse
https://soundcloud.com/kokiri-music/kokiri-other-side-of-love?in=digitalstreams/sets/sexydeephouse
https://soundcloud.com/kaytranada/teedra-moses-be-your-girl?in=digitalstreams/sets/sexydeep

https://soundcloud.com/aka-aka/aka-aka-moguai-satisfied-club-mix?in=digitalstreams/sets/edmhits
https://soundcloud.com/eric-prydz/trubble?in=digitalstreams/sets/edmhits
https://soundcloud.com/dukedumont/inhale?in=digitalstreams/sets/edmhits
https://soundcloud.com/martinsolveig/my-love?in=digitalstreams/sets/edmhits
https://soundcloud.com/davidguetta/david-guetta-martin-garrix-brooks-like-i-do?in=digitalstreams/sets/edmhits
https://soundcloud.com/bakermatmusic/bakermatbaby?in=digitalstreams/sets/edmhits
https://soundcloud.com/troyboi/troyboi-x-ekali-truth?in=digitalstreams/sets/edmhits
https://soundcloud.com/musical-freedom/coming-home?in=digitalstreams/sets/edmhits
https://soundcloud.com/diplo/diplo-french-montana-lil-pump?in=digitalstreams/sets/edmhits
https://soundcloud.com/drumcode/ramiro-lopez-being-alone?in=digitalstreams/sets/edmhits
https://soundcloud.com/dukedumont/runway-for-club-play-only-pt-5?in=digitalstreams/sets/edmhits
https://soundcloud.com/phantoms/pulling-me-in-1?in=d

...

[          ] 0%  '%'https://soundcloud.com/flume/disclosure-you-me-flume-remix?in=ilyanaazman/sets/insanity
https://soundcloud.com/djvanic/fucking-crazy-vanic-remix?in=ilyanaazman/sets/insanity
https://soundcloud.com/thesquidiot/disenchantment-theme-remix?in=ilyanaazman/sets/insanity
https://soundcloud.com/tribaltrapmusic/lost?in=ilyanaazman/sets/insanity
https://soundcloud.com/yume-network/go-shina-kamado-tanjiro-no-uta-feat-nami-nami-nakagawa-ckody-remix-demon-slayer?in=ilyanaazman/sets/insanity
https://soundcloud.com/thesquidiot/s3rl-mtc-tifwhitney-remix?in=ilyanaazman/sets/insanity
https://soundcloud.com/djvanic/skinny-love-vanic?in=ilyanaazman/sets/insanity
https://soundcloud.com/zedsdead/marina-and-the-diamonds-lies?in=ilyanaazman/sets/insanity
https://soundcloud.com/pegboardnerds/pegboard-nerds-hero-da-tweekaz-guitar-hero-mix?in=ilyanaazman/sets/insanity
https://soundcloud.com/mrsuicidesheep/major-lazer-be-together-ft-wild-bellevanic-remix?in=ilyanaazman/sets/insanity
https://so

https://soundcloud.com/dirtychad/summer-15-konoha-peace-trap-remix?in=ilyanaazman/sets/naruto
https://soundcloud.com/rifti-beats/naruto-those-who-inherit-the-will-of-fire-lofi-hip-hop-remix?in=ilyanaazman/sets/naruto
https://soundcloud.com/kaidragan/amv-naruto-shippuden-samidare-ksolis-trap-remix?in=ilyanaazman/sets/naruto
https://soundcloud.com/dirty-kid-basel-1/dirty-kid-naruto-ost-utsusemi?in=ilyanaazman/sets/naruto
https://soundcloud.com/haardtek/downfall?in=ilyanaazman/sets/naruto
https://soundcloud.com/desanctus/obitorin?in=ilyanaazman/sets/naruto
https://soundcloud.com/lilstupidmane/rip-obito?in=ilyanaazman/sets/naruto
https://soundcloud.com/lsbbeats/naruto-shippuden-samidare-lsb-remix?in=ilyanaazman/sets/naruto
https://soundcloud.com/odece/naruto-shippuden-samidare-odece-trap-remix?in=ilyanaazman/sets/naruto
https://soundcloud.com/itsrudeboy/naruto-yamagsumi-rude-remix?in=ilyanaazman/sets/naruto
https://soundcloud.com/lsbbeats/naruto-shippuden-ost-iii-obitos-theme-lsb-remix?in=

https://soundcloud.com/istrapkingdom/postmalone-rockstar-crankdat-remix?in=ilyanaazman/sets/bestoftrapnation
https://soundcloud.com/besomorph/cradless?in=ilyanaazman/sets/bestoftrapnation
https://soundcloud.com/ka1eofficial/roll-the-dice?in=ilyanaazman/sets/bestoftrapnation
https://soundcloud.com/trapicalmusic/on-and-on?in=ilyanaazman/sets/bestoftrapnation
https://soundcloud.com/kshmr/sounds-of-kshmr-vol-2-out-now?in=ilyanaazman/sets/bestoftrapnation
https://soundcloud.com/itsdifferent/fashionweek?in=ilyanaazman/sets/bestoftrapnation
https://soundcloud.com/djseanbobomashups/diplo-revolution-sean-bobo?in=ilyanaazman/sets/bestoftrapnation
https://soundcloud.com/cremesongs/all-talk?in=ilyanaazman/sets/bestoftrapnation
https://soundcloud.com/shadowanthem/maroon-5-dont-wanna-know?in=ilyanaazman/sets/bestoftrapnation
https://soundcloud.com/hoobermusic/x-ambassadors-renegades-hoober-remix-1?in=ilyanaazman/sets/bestoftrapnation
https://soundcloud.com/itstrapnetwork/rain-man-feat-oly-bring-back

https://soundcloud.com/odece/betrayal-3?in=ilyanaazman/sets/anime-trap
https://soundcloud.com/rifti-beats/attack-on-titan-season-4-eren-vs-jaw-titan-nutcracker-theme-trap-remix?in=ilyanaazman/sets/anime-trap
https://soundcloud.com/odece/betrayal-2?in=ilyanaazman/sets/anime-trap
https://soundcloud.com/shibuisaight/aot-betrayal-odece-remix?in=ilyanaazman/sets/anime-trap
https://soundcloud.com/rifti-beats/attack-on-titan-shingeki-no-kyojin-ost-vogel-im-kafig-hip-hop-remix?in=ilyanaazman/sets/anime-trap
https://soundcloud.com/rifti-beats/fullmetal-alchemist-brotherhood-ost-trishas-lullaby-hip-hop-remix?in=ilyanaazman/sets/anime-trap
https://soundcloud.com/drsparxbeats/my-war-attack-on-titan-season-4-openinghip-hop-remix?in=ilyanaazman/sets/anime-trap
https://soundcloud.com/odece/awakening?in=ilyanaazman/sets/anime-trap
https://soundcloud.com/thelasthumanwallet/madara-uchiha-the-god-awakened-axhel-remix?in=ilyanaazman/sets/anime-trap
https://soundcloud.com/lucas-fader/naurto-ost-girei-lucas

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(set_playlists, columns=['User ID',
                                          'User name',
                                          'Playlist name',
                                          'Track count',
                                          'Create at',
                                          'Likes count',
                                          'Reposts count',
                                          'Track ID list'
                                         ])
df.to_csv('playlist_HTML.csv', index=False, sep='\t', encoding='utf-8')